In [73]:
# code and data: https://github.com/rpeden/cat-or-not/releases
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from PIL import Image
from random import shuffle, choice
import numpy as np
import os

In [74]:
IMAGE_SIZE = 256
IMAGE_DIRECTORY = './data/training_set'

In [75]:
def label_img(name):
    if name == 'cats': return np.array([1, 0])
    elif name == 'notcats' : return np.array([0, 1])

In [76]:
def load_data(IMAGE_DIRECTORY):
    print("Loading images...")
    train_data = []
    directories = next(os.walk(IMAGE_DIRECTORY))[1]

    for dirname in directories:
        print("Loading {0}".format(dirname))
        file_names = next(os.walk(os.path.join(IMAGE_DIRECTORY, dirname)))[2]
        for i in len(file_names):
            image_name = choice(file_names)
            image_path = os.path.join(IMAGE_DIRECTORY, dirname, image_name)
            label = label_img(dirname)
            if "DS_Store" not in image_path:
                img = Image.open(image_path)
                img = img.convert('L')
                img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
                train_data.append([np.array(img)/255, label])
    return train_data

In [77]:
training_data = load_data(IMAGE_DIRECTORY)
training_images = np.array([i[0] for i in training_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
training_labels = np.array([i[1] for i in training_data]) #.reshape(-1, 1)

Loading images...
Loading cats
Loading notcats


In [78]:
training_images.shape, training_labels.shape

((399, 256, 256, 1), (399, 2))

In [79]:
# 轉為 Dataset，含 X/Y 資料
BATCH_SIZE = 32
# 一定要加 .batch(BATCH_SIZE)，否則餵入模型會出錯，因為筆數會放在最後一維
# 使用所有檔案訓練，GPU記憶體還是不足
# error message：Failed copying input tensor from device:CPU:0 to device:GPU:0 in order to run _EagerConst
train_ds = tf.data.Dataset.from_tensor_slices((training_images, training_labels)).batch(BATCH_SIZE)

In [80]:
train_ds = train_ds.prefetch(buffer_size=BATCH_SIZE).cache()

In [81]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', 
                     input_shape=training_images.shape[1:]))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation = 'sigmoid'))

    return model

In [82]:
print('creating model')
model = create_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('training model')
model.fit(train_ds, epochs=10, verbose=1)
# model.save("model.h5")

creating model
training model
Epoch 1/10
13/13 [==============================] - 3s 136ms/step - loss: 1.5925 - accuracy: 0.4286
Epoch 2/10
13/13 [==============================] - 2s 135ms/step - loss: 1.1206 - accuracy: 0.4361
Epoch 3/10
13/13 [==============================] - 2s 135ms/step - loss: 0.8997 - accuracy: 0.5238
Epoch 4/10
13/13 [==============================] - 2s 136ms/step - loss: 0.6093 - accuracy: 0.6742
Epoch 5/10
13/13 [==============================] - 2s 135ms/step - loss: 0.4196 - accuracy: 0.8095
Epoch 6/10
13/13 [==============================] - 2s 135ms/step - loss: 0.2730 - accuracy: 0.9073
Epoch 7/10
13/13 [==============================] - 2s 135ms/step - loss: 0.2104 - accuracy: 0.9373
Epoch 8/10
13/13 [==============================] - 2s 135ms/step - loss: 0.1208 - accuracy: 0.9724
Epoch 9/10
13/13 [==============================] - 2s 137ms/step - loss: 0.0782 - accuracy: 0.9950
Epoch 10/10
13/13 [==============================] - 2s 137ms/step - l

In [83]:
IMAGE_DIRECTORY = './data/test_set'
test_data = load_data(IMAGE_DIRECTORY)
test_images = np.array([i[0] for i in test_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
test_labels = np.array([i[1] for i in test_data]) #.reshape(-1, 1)

Loading images...
Loading cats
Loading notcats


In [84]:
test_images.shape, test_labels.shape

((397, 256, 256, 1), (397, 2))

In [85]:
# 轉為 Dataset，含 X/Y 資料
test_data = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(BATCH_SIZE)

In [96]:
loss, acc = model.evaluate(test_images, test_labels, verbose=1)
print("accuracy: {0}".format(acc * 100))

13/13 [==============================] - 1s 42ms/step - loss: 1.4258 - accuracy: 0.4962
accuracy: 49.6221661567688


In [97]:
pred = np.argmax(model.predict(test_images), axis=-1)
np.count_nonzero(pred == test_labels[:, 0].reshape(-1)) / len(pred)

13/13 [==============================] - 0s 38ms/step


0.5037783375314862